In [1]:
import pandas as pd
import numpy as np

In [2]:
bank_df = pd.read_csv("./data/bank_accounts.csv")
bank_df

/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,userid,bank_account
0,21829134,923302000003892
1,95910542,11002023212822
2,96941876,189303223
3,23452396,2280003199803
4,12647942,32002028484803
...,...,...
350836,23568930,084902023320802
350837,17207934,023102020991822
350838,23862272,0090343142
350839,23983072,3330993800


In [3]:
cc_df = pd.read_csv("./data/credit_cards.csv")
cc_df

,userid,credit_card
0,2579938,832299xxxxxx4902|02-2019
1,2154902,322429xxxxxx3848|05-2020
2,5578604,322429xxxxxx1293|11-2016
3,2051728,388329xxxxxx4303|09-2016
4,6216212,322421xxxxxx4238|05-2018
...,...,...
38703,25879226,839199xxxxxx3838|12-2022
38704,9331492,398138xxxxxx2208|08-2019
38705,2890486,839330xxxxxx9180|08-2022
38706,107429252,899933xxxxxx2930|06-2021


In [4]:
dev_df = pd.read_csv("./data/devices.csv")
dev_df

,userid,device
0,10499978,0Zu/mWZ4cFsvobYglgZEc5VSxwwajRwrm74pBuwnrTu=
1,54526742,zBMRKyz98iy5Z7sh+JIOdF612J1CB8ggvFBI6MP361q=
2,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=
3,55020986,anRXS7+zVgCGKz5C9X3dzewIhSdojGEi5TO07pb2y+K=
4,100471502,hbIARr+USwvjjzc8QUVtD5a/apR1DJX6P3fYvgDL4gW=
...,...,...
1430306,7061456,ZSTXD6wfHN/1qPVqMrIbwHYX5ELWlUPO3NFXCHK50Mc=
1430307,26416970,o+9gbX201ITS3MuMZYjuZUzV7sOoR57/0ttyX3eSWTA=
1430308,79633116,NZhNiXYhArpPquzrqh/kVj2IZ3QPRKm6FH90vsTlhiy=
1430309,19883370,dZ7YqVTprtTvLT4qS0YEWsn9EJahBJVQtTLKC6WaHZA=


In [5]:
orders_df = pd.read_csv("./data/orders.csv")
orders_df

,orderid,buyer_userid,seller_userid
0,1953278092,47388162,20822974
1,1953295120,26855196,16416890
2,1953306402,121296714,28477978
3,1953314712,131221930,72837912
4,1953381964,183398314,28423332
...,...,...,...
620942,1956636054,14222102,70940136
620943,1956642760,2978154,60766142
620944,1956651496,221974772,15541938
620945,1956660812,152533590,80986700


In [6]:
linked_users_dict = dict()
linked_users_dict

{}

In [14]:
# Find direct links by bank account
for label, group in bank_df.groupby("bank_account"):
    for i in group["userid"]:
        for j in group["userid"]:
            if i not in linked_users_dict:
                linked_users_dict[i] = set()
            linked_users_dict[i].add(j)

In [15]:
# Find direct links by credit card
for label, group in cc_df.groupby("credit_card"):
    for i in group["userid"]:
        for j in group["userid"]:
            if i not in linked_users_dict:
                linked_users_dict[i] = set()
            linked_users_dict[i].add(j)

In [16]:
# Find direct links by device
for label, group in dev_df.groupby("device"):
    for i in group["userid"]:
        for j in group["userid"]:
            if i not in linked_users_dict:
                linked_users_dict[i] = set()
            linked_users_dict[i].add(j)

In [17]:
def is_fraud(user_a, user_b):
    visited = set()
    
    def inner(a, b):
        visited.add(a)
        if a not in linked_users_dict or b not in linked_users_dict:
            return 0
        elif b in linked_users_dict[a]:
            return 1
        else:
            output = 0
            for c in linked_users_dict[a]:
                if c in visited:
                    continue
                output = output or inner(c, b)
            return output
        
    return inner(user_a, user_b)

In [18]:
output_df = orders_df.copy()
output_df["is_fraud"] = output_df.apply(
    lambda row: is_fraud(row["buyer_userid"], row["seller_userid"]), axis = 1
)
output_df

,orderid,buyer_userid,seller_userid,is_fraud
0,1953278092,47388162,20822974,0
1,1953295120,26855196,16416890,0
2,1953306402,121296714,28477978,0
3,1953314712,131221930,72837912,0
4,1953381964,183398314,28423332,0
...,...,...,...,...
620942,1956636054,14222102,70940136,0
620943,1956642760,2978154,60766142,0
620944,1956651496,221974772,15541938,0
620945,1956660812,152533590,80986700,0


In [19]:
output_df[output_df["is_fraud"] == 1]

,orderid,buyer_userid,seller_userid,is_fraud
1649,1954198318,221232712,66353306,1
2679,1955598428,35545436,70763052,1
3545,1954515646,32834366,188151804,1
5938,1953728724,168491444,158559422,1
9708,1953543830,223406364,193350172,1
...,...,...,...,...
612805,1955929340,235579716,80629986,1
614563,1954626246,197784360,12132476,1
616320,1953812320,235299524,229915488,1
616725,1954821958,111271962,66519400,1


In [20]:
save_df = output_df.copy()
save_df = save_df.drop("buyer_userid", axis = 1)
save_df = save_df.drop("seller_userid", axis = 1)
save_df.to_csv("./data/output.csv", index = False)
save_df

,orderid,is_fraud
0,1953278092,0
1,1953295120,0
2,1953306402,0
3,1953314712,0
4,1953381964,0
...,...,...
620942,1956636054,0
620943,1956642760,0
620944,1956651496,0
620945,1956660812,0
